# Volume 1, Chapter 6: Structured Outputs

**Get JSON instead of text using Pydantic**

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/eduardd76/AI_for_networking_and_security_engineers/blob/main/CODE/Colab-Notebooks/Vol1_Ch6_Structured_Outputs.ipynb)

In [ ]:
!pip install -q langchain langchain-anthropic pydantic
import os
from getpass import getpass
if 'ANTHROPIC_API_KEY' not in os.environ:
    os.environ['ANTHROPIC_API_KEY'] = getpass('Enter API key: ')
print("✓ Ready!")

## Example 1: Parse Interface Config

In [ ]:
from langchain_anthropic import ChatAnthropic
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
from pydantic import BaseModel, Field
from typing import Optional

class InterfaceInfo(BaseModel):
    name: str = Field(description="Interface name")
    ip_address: Optional[str] = Field(description="IP address with mask")
    description: Optional[str] = Field(description="Description")
    status: str = Field(description="Admin status")

llm = ChatAnthropic(model="claude-3-5-sonnet-20241022", temperature=0)
parser = PydanticOutputParser(pydantic_object=InterfaceInfo)

config = """interface GigabitEthernet0/1
 description WAN_UPLINK
 ip address 203.0.113.1 255.255.255.252
 no shutdown"""

prompt = PromptTemplate(
    template="Extract interface info from:\n{config}\n\n{format_instructions}",
    input_variables=["config"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

response = llm.invoke(prompt.format(config=config))
interface = parser.parse(response.content)

print(f"Name: {interface.name}")
print(f"IP: {interface.ip_address}")
print(f"Description: {interface.description}")
print(f"Status: {interface.status}")

## Example 2: Security Analysis

In [ ]:
from enum import Enum
from typing import List

class Severity(str, Enum):
    CRITICAL = "critical"
    HIGH = "high"
    MEDIUM = "medium"
    LOW = "low"

class Issue(BaseModel):
    severity: Severity
    issue: str
    location: str
    fix: str

class Analysis(BaseModel):
    issues: List[Issue]
    risk_score: int

parser = PydanticOutputParser(pydantic_object=Analysis)

config = """line vty 0 4
 transport input telnet
 password cisco123

snmp-server community public RO"""

prompt = PromptTemplate(
    template="Analyze for security issues:\n{config}\n\n{format_instructions}",
    input_variables=["config"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

response = llm.invoke(prompt.format(config=config))
analysis = parser.parse(response.content)

print(f"Risk Score: {analysis.risk_score}/10\n")
for i, issue in enumerate(analysis.issues, 1):
    print(f"{i}. [{issue.severity.value.upper()}] {issue.issue}")
    print(f"   Location: {issue.location}")
    print(f"   Fix: {issue.fix}\n")

## 🎯 Key Benefits

- ✅ Consistent JSON format
- ✅ Automatic type validation
- ✅ Easy to integrate with code
- ✅ No manual parsing needed